# Multimodal RAG

In [1]:
import os
import sys
import glob

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from dotenv import load_dotenv
load_dotenv(os.path.join(repo_dir,'.env'))

import requests
import json
import base64
from pprint import pprint

## utils

In [2]:
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        image_binary = image_file.read()
        base64_image = base64.b64encode(image_binary).decode()
        return base64_image

## Multimodal call

### sambastudio llava call method

In [3]:
# sambastudio call
def llava_call(prompt, image_path):
    image=image_to_base64(image_path)
    endpoint_url = f"{os.environ.get('LVLM_BASE_URL')}/api/predict/generic/{os.environ.get('LVLM_PROJECT_ID')}/{os.environ.get('LVLM_ENDPOINT_ID')}"
    endpoint_key = os.environ.get('LVLM_API_KEY')
    # Define the data payload
    data = {
        "instances": [{
            "prompt": prompt,
            "image_content": f"{image}"
        }],
        "params": {
            "do_sample": {"type": "bool", "value": "false"},
            "max_tokens_to_generate": {"type": "int", "value": "512"},
            "temperature": {"type": "float", "value": "1"},
            "top_k": {"type": "int", "value": "50"},
            "top_logprobs": {"type": "int", "value": "0"},
            "top_p": {"type": "float", "value": "1"}
        }
    }
    # Define headers
    headers = {
        "Content-Type": "application/json",
        "key": endpoint_key
    }
    response = requests.post(endpoint_url, headers=headers, data=json.dumps(data))
    return response.json()["predictions"][0]['completion']

### QA Llava Call

In [4]:
prompt = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the humans question. USER: <image>\nhow many birds could you find at 4pm?. ASSISTANT:"
image_path = os.path.join(kit_dir,"data","sample_docs","sample.png")
llava_call(prompt, image_path)

'At 4 pm, you could find approximately 10 birds on the tree.'

### Summary Llava call

In [5]:
prompt = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the humans question. USER: <image>\nDescribe the image in detail. Be specific about graphs, such as bar plots, scatter plots, or others. ASSISTANT:"
llava_call(prompt, image_path)

'The image displays a graph showing the number of birds on a tree at different times of the day. The graph is a combination of a bar plot and a scatter plot, with the bar plot showing the number of birds at various times of the day, and the scatter plot showing the number of birds on a tree at a specific time.\n\nThe graph is divided into two main sections. The first section, which is the bar plot, shows the number of birds on a tree at different times of the day, with the bars extending from 10 am to 11 pm. The second section, which is the scatter plot, shows the number of birds on a tree at a specific time, with the x-axis representing the time and the y-axis representing the number of birds.\n\nThe graph is labeled with the time of the day, and the number of birds is represented by the number of orange dots on the graph. The dots are scattered throughout the graph, with some appearing closer to the bars and others appearing closer to the scatter plot.'

## Doc Extraction

### Unstructured PDF extraction

In [6]:
from unstructured.partition.pdf import partition_pdf

# Path to save images
file_path=os.path.join(kit_dir, "data", "sample_docs", "invoicesample.pdf")
output_path=os.path.splitext(file_path)[0]

# Get elements
raw_pdf_elements = partition_pdf(
    filename=file_path,
    extract_images_in_pdf=True,
    strategy='hi_res',
     hi_res_model_name="yolox",
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=1000,
    new_after_n_chars=800,
    combine_text_under_n_chars=500,
    extract_image_block_output_dir=output_path,
)

/Users/luiss/.pyenv/versions/3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection

### View Elements

In [7]:
for i, element in enumerate(raw_pdf_elements):
    print(f"\033[95m ELEMENT {i}\033[00m")
    print(f"TYPE: {type(element)}")
    print(f"META: {element.metadata.to_dict()}")
    print(f"TEXT: {element.text}")
    print("\n\n##########\n")

 ELEMENT 0
TYPE: <class 'unstructured.documents.elements.CompositeElement'>
META: {'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-07-29T09:07:11', 'page_number': 1, 'orig_elements': 'eJy9lk2P2zYQhv8KofbQAqbE74+9FU1R7CGbFOv0slgYlDjyMpElQ6J2swn630tJTpEmToAYsI/zaoYi55kZ8u5jBg3soI2b4LMrlGmrnClrgZmpFRaWeVxy6bFlglIjjK6FzFYo20F03kWXYj5mVdf1PrQuwjDbjXvuxrh5gLB9iElhnNsUc5Cfgo8PSaVK8qTuu9DGKe7uzlqWyxWiQuTifoU+2YyqXE42lUTn7IiwRCQlG56HCLvpJK/De2hu966C7J/0wUOEKoau3VSNG4bNvu/K5EZyprhUyaEODcTnPcyxr19m84bb7ei286nuMmi32f2sDnGz63yoA8w5Y4QJTDRmdk3sFdFXlE7R+xS5acddCf102mkTEd5P+ch+W6//uFlfv7pB61eT66f/rkNs5u1+iaVmVkghDRZl7bBgssaloR7XRCpDLSesYmfDQrXMdcq6JDmds77YSsqcT7ahYsL0pb34nwbFcPUNJmE3pXXv5r1mxZsB+qFoxjAMxYuuGqecDUVa521aeChcwEN0fYQevwux2I1NDAmdazbv2u6pAZ/W6iH2AR6hL6bMFYPb7RvY+K4aitA+dqGCRSrqsB17wBTT/O1+e9YC+bworqcTHysKaakX1jrMjaepMqDGprSAneSqFKT0lTHnKwpC5yrgjC/U/xOUys0scKFye0yYQ07tVkbkhbv1BbTtM/pzbN2Taz8nc+P63sXwCOvJ8wihEsBJYkusjLNYKFFhqwhglfJpBBDg4qyETE5XSKiUtgOhWZBcHJBxJqdO

In [8]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
# TableChunk if Table > max chars set above
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 2,
 "<class 'unstructured.documents.elements.Table'>": 1}

In [9]:
from langchain.schema import Document


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        meta = element.metadata.to_dict()
        meta["type"] = "table"
        categorized_elements.append(Document(page_content=element.metadata.text_as_html, metadata=meta))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        meta = element.metadata.to_dict()
        meta["type"] = "text"
        categorized_elements.append(Document(page_content=str(element), metadata=meta))

# Tables
table_docs = [e for e in categorized_elements if e.metadata["type"] == "table"]
print(len(table_docs))

# Text
text_docs = [e for e in categorized_elements if e.metadata["type"] == "text"]
print(len(text_docs))

1
2


### Text and table summaries

In [10]:
from utils.model_wrappers.langchain_llms import SambaNovaFastAPI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import load_prompt

In [11]:
text_prompt = load_prompt(os.path.join(kit_dir, "prompts", "llama70b-text_summary.yaml"))
table_prompt = load_prompt(os.path.join(kit_dir, "prompts", "llama70b-table_summary.yaml"))

# Summary chain
model = SambaNovaFastAPI(
    max_tokens = 500,
    model= 'llama3-8b',
)
text_summarize_chain = {"element": lambda x: x} | text_prompt | model | StrOutputParser()
table_summarize_chain = {"element": lambda x: x} | table_prompt | model | StrOutputParser()

### Text Summaries

In [12]:
# Apply to text
texts = [i.page_content for i in text_docs if i.page_content != ""]
if texts:
    text_summaries = text_summarize_chain.batch(texts, {"max_concurrency": 1})

In [13]:
text_summaries

['Here is a concise summary of the text chunk:\n\nThis is an invoice addressed to Denny Gunawan, with a total amount of $39.60, and details of two addresses in Melbourne, VIC.',
 'A receipt summary:\n\nA subtotal of $36.00 is calculated, then a 10% GST (Goods and Services Tax) is added, resulting in a total of $39.60.']

### Table summaries

In [14]:
# Apply to tables
tables = [i.page_content for i in table_docs]
if tables:
    table_summaries = table_summarize_chain.batch(tables, {"max_concurrency":1})

In [15]:
table_summaries

['Here is a concise summary of the table:\n\n**Total Items:** 9\n**Total Cost:** $47.34']

### Image summary

In [16]:
image_prompt = "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the humans question. USER: <image>\n, such as bar plots. ASSISTANT:"
image_prompt = load_prompt(os.path.join(kit_dir, "prompts", "llava.yaml"))
prompt = image_prompt.format(instruction = "Describe the image in detail. Be specific about graphs include name of axis, labels, legends and important numerical information")
image_paths = []
image_paths.extend(glob.glob(os.path.join(output_path, '*.jpg')))
image_paths.extend(glob.glob(os.path.join(output_path, '*.png')))

image_summaries = []
image_docs = []

for image_path in image_paths:
    result = llava_call(prompt, image_path)
    image_summaries.append(result)
    image_docs.append(Document(
                        page_content=result, 
                        metadata={
                                'type': 'image', 
                                'file_directory': os.path.dirname(image_path),
                                'filename': os.path.basename(image_path),
                            }
                        )
                    )

In [17]:
image_summaries

["The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is simple yet effective in conveying the company's message."]

In [18]:
image_docs

[Document(metadata={'type': 'image', 'file_directory': '/Users/luiss/Documents/projects/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}, page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is simple yet effective in conveying the company's message.")]

### add to vectorstore

In [19]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from utils.model_wrappers.langchain_embeddings import SambaStudioEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries", embedding_function=SambaStudioEmbeddings(
        type="sambastudio", batch_size=1, coe=True, select_expert="e5-mistral-7b-instruct")
)

# The storage layer for the parent documents
store = InMemoryByteStore()  
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k":2}
)

/Users/luiss/.pyenv/versions/3.11.3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
# Add texts
if texts:
    doc_ids = [str(uuid.uuid4()) for _ in text_docs]
    summary_texts = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(text_summaries)
    ]
    retriever.vectorstore.add_documents(summary_texts)
    retriever.docstore.mset(list(zip(doc_ids, text_docs)))

# Add tables
if tables:
    table_ids = [str(uuid.uuid4()) for _ in table_docs]
    summary_tables = [
        Document(page_content=s, metadata={id_key: table_ids[i]})
        for i, s in enumerate(table_summaries)
    ]
    retriever.vectorstore.add_documents(summary_tables)
    retriever.docstore.mset(list(zip(table_ids, table_docs)))

# Add images
if image_summaries:
    img_ids = [str(uuid.uuid4()) for _ in image_summaries]
    summary_img = [
        Document(page_content=s, metadata={id_key: img_ids[i]})
        for i, s in enumerate(image_summaries)
    ]
    retriever.vectorstore.add_documents(summary_img)
    retriever.docstore.mset(
        list(zip(img_ids, image_docs))
    )  # Store the image summary as the raw document

In [ ]:
retriever.invoke("what is the final price in the invoice?")

In [22]:
retriever.invoke("what is the logo of the company")

[Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is simple yet effective in conveying the company's message.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
 Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.

## Retrieval with raw text, raw tables and image summaries

In [23]:
from langchain.chains import RetrievalQA

prompt = load_prompt(os.path.join(kit_dir,"prompts","llama70b-knowledge_retriever_custom_qa_prompt.yaml"))

chain = RetrievalQA.from_llm(
    llm = model,
    retriever=retriever,
    return_source_documents=True,
    input_key="question",
    output_key="answer"
)
chain.combine_documents_chain.llm_chain.prompt=prompt


In [24]:
chain.invoke({"question": "what is the final price in the invoice?"})

{'question': 'what is the final price in the invoice?',
 'answer': ' Sure, I can help you with that! Based on the provided context, the final price in the invoice is $39.60. This can beWiki confirmed by looking at the "Total" section of the invoice, which states $39.60.\n\nHere\'s a breakdown of how I arrived at this answer:\n\n* I first looked at the "Subtotal" section, which states $36.00.\n* Next, I looked at the "GST (10%)" section, which states $3.60.\n* Finally, I added the subtotal and GST amounts together to get the final total, which is $39.60.\n\nI hope that helps! Let me know if you have any other questions.',
 'source_documents': [Document(page_content='THANK YOU\n\n* Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales dapibus fermentum. Nunc adipiscing, magna sed scelerisque cursus, erat lectus dapibus urna, sed facilisis leo dui et ipsum.\n\nSubtotal | Total\n\nSubtotal\n\nGST (10%)\n\nTotal\n\n$36.00\n\n$3.60\n\n$39.60', metadata={'filetype': 'applic

In [25]:
chain.invoke("what is the logo of the company")

{'question': 'what is the logo of the company',
 'answer': ' Based on the provided context, the logo of the company is a gold and yellow logo featuring a sun in the center, surrounded by trees. The sun is positioned at the top left corner, while the trees are located at the bottom right corner. The logo is placed on a white background, making it stand out and easily recognizable.',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is 

## Retrieval with raw text, raw tables and raw images

In [26]:
query = "what it says the text in the logo"#"what is the logo of the company?"

In [27]:
retriever.invoke(query)

[Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is simple yet effective in conveying the company's message.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
 Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.

In [28]:
chain.invoke({"question": query})

{'question': 'what it says the text in the logo',
 'answer': ' The text in the logo of Sunny Farm, as described in the first context, does not explicitly state what the text says. However, based on the description of the logo, it appears to feature the company name "Sunny Farm" in a gold and yellow color scheme, with a sun in the center and trees surrounding it.\n\nTherefore, a possible answer to the question "what does the text in the logo say?" could be:\n\n"The text in the logo of Sunny Farm appears to be the company name \'Sunny Farm\', written in a gold and yellow color scheme with a sun in the center and trees surrounding it."\n\nPlease note that this answer is based on the information provided in the context and may not be entirely accurate. If further information is available or if the question is ambiguous, it\'s always best to clarify or ask for additional details.',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that speciali

### filter image results

In [29]:
def get_retrieved_images(retriever, query):
    results=retriever.invoke(query)
    results = [result for result in results if result.metadata["type"]=="image"]
    return results

In [30]:
retrieved_images = get_retrieved_images(retriever, query)
retrieved_images

[Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center, symbolizing the warmth and freshness of the products. The sun is surrounded by trees, which further emphasize the connection to nature and the source of the fresh produce.\n\nThe logo is placed on a white background, making it stand out and be easily recognizable. The sun in the logo is positioned at the top left corner, while the trees are located at the bottom right corner. The overall design of the logo is simple yet effective in conveying the company's message.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'})]

### Generate response over retrieved raw images 

In [31]:
def get_image_answers(retrieved_image_docs, query):
    image_answer_prompt_template = load_prompt(os.path.join(kit_dir,"prompts","llava-qa.yaml"))
    image_answer_prompt = image_answer_prompt_template.format(question = query)
    answers = []
    for doc in retrieved_image_docs:
        image_path = os.path.join(doc.metadata["file_directory"],doc.metadata["filename"])
        answers.append(llava_call(image_answer_prompt, image_path))
    return answers

In [32]:
image_answers = get_image_answers(retrieved_images, query)
image_answers

['The text in the logo says "Sunny Farm Victoria."']

In [33]:
def get_retrieved_docs(retriever, query):
    results=retriever.invoke(query)
    results = [result for result in results if result.metadata["type"]!="image"]
    return results

In [34]:
context_docs = get_retrieved_docs(retriever, query)
context_docs

[Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.60\n\nInvoice Number: #20130304', metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-05-08T08:57:57', 'page_number': 1, 'orig_elements': 'eJy9ltuO2zYQhl9FUHvRAqbE82HviqYo9iKbFOv0ZrEQKHHkZVaWBIleZxP03UtJTpCDEyAGbMA382vGIv9vhtTdhxQa2EIbCu/SqyQ1qqTUuhJhRQTinBpkwXJky9rVdSV5xWS6StItBOtssLHmQ1p13eB8awOMc9zY524Xigfwm4cQFcqYiTUHee9deIgqkYJFte98G6a6uztjaCZWCeE84/er5GNMiczEFBOBVUaPCEtFVNLxeQywnXby2r+D5ra3FaT/xQcOAlTBd21RNXYci37oypiGMyqZkDGh9g2E5x7m2tcv03nB7WZnN/Ou7lJoN+n9rI6h2HbO1x5mzyimHGGBsF5jfSVU/E3Vfaws2t22hGHa7bSIAO8mP9I/1uu/btbXr26S9asp9eN71z4083K/xmJVSZQRHAHnDHGHS2Sto0hVXNRQSoHJ+bAQJTIVXRc4I7PrSyyFyNgUa8InTF/HS/5pUDST32Hit5OtvZ3XmuZvRhjG/G03bKDPX3TVbjJtzO34WPS7svFV0e3b3Ho0BjsEGNCjD/l21wQfCdqmeGy7fQMu/uUAYfDwBEM+GZiPdts3ULiuGnPfPnW+gkXKa7/ZDYAIItnbfnPWPvm8N66njR/rDai0AWIM4ix2BI+dgGytCcK1KSVIyqjh5+sNTOZmYJQt

In [37]:
prompt = load_prompt(os.path.join(kit_dir, "prompts", "llama70b-knowledge_retriever_custom_qa_prompt.yaml"))
text_contexts = [doc.page_content for doc in context_docs]
full_context = '\n\n'.join(image_answers)+'\n\n'+'\n\n'.join(text_contexts)
formated_prompt = prompt.format(context=full_context, question=query)
formated_prompt
model.invoke(formated_prompt)

' Sure, I can help you with that! Based on the provided context, the text in the logo says "Sunny Farm Victoria."'